In [ ]:
#
# Google Colab setup
#
#!pip install git+https://github.com/doctorado-ml/stree

In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification, load_iris, load_wine
from sklearn.model_selection import train_test_split
from stree import Stree
import time

In [ ]:
import os
if not os.path.isfile('data/creditcard.csv'):
    !wget --no-check-certificate --content-disposition http://nube.jccm.es/index.php/s/Zs7SYtZQJ3RQ2H2/download
    !tar xzf creditcard.tgz

In [2]:
random_state=1

def load_creditcard(n_examples=0):
    import pandas as pd
    import numpy as np
    import random
    df = pd.read_csv('data/creditcard.csv')
    print("Fraud: {0:.3f}% {1}".format(df.Class[df.Class == 1].count()*100/df.shape[0], df.Class[df.Class == 1].count()))
    print("Valid: {0:.3f}% {1}".format(df.Class[df.Class == 0].count()*100/df.shape[0], df.Class[df.Class == 0].count()))
    y = df.Class
    X = df.drop(['Class', 'Time', 'Amount'], axis=1).values
    if n_examples > 0:
        # Take first n_examples samples
        X = X[:n_examples, :]
        y = y[:n_examples, :]
    else:
        # Take all the positive samples with a number of random negatives
        if n_examples < 0:
            Xt = X[(y == 1).ravel()]
            yt = y[(y == 1).ravel()]
            indices = random.sample(range(X.shape[0]), -1 * n_examples)
            X = np.append(Xt, X[indices], axis=0)
            y = np.append(yt, y[indices], axis=0)
    print("X.shape", X.shape, " y.shape", y.shape)
    print("Fraud: {0:.3f}% {1}".format(len(y[y == 1])*100/X.shape[0], len(y[y == 1])))
    print("Valid: {0:.3f}% {1}".format(len(y[y == 0]) * 100 / X.shape[0], len(y[y == 0])))
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=random_state, stratify=y)
    return Xtrain, Xtest, ytrain, ytest

# data = load_creditcard(-5000) # Take all true samples + 5000 of the others
# data = load_creditcard(5000)  # Take the first 5000 samples
data = load_creditcard(-1000) # Take all the samples

Xtrain = data[0]
Xtest = data[1]
ytrain = data[2]
ytest = data[3]
# Set weights inverse to its count class in dataset
weights = np.ones(Xtrain.shape[0],) * 1.00244
weights[ytrain==1] = 1.99755 

Fraud: 0.244% 196
Valid: 99.755% 80234
X.shape (1196, 28)  y.shape (1196,)
Fraud: 16.472% 197
Valid: 83.528% 999


In [3]:
C = 23
print("Accuracy of Train without weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain).score(Xtrain, ytrain))
print("Accuracy of Train with    weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain, sample_weight=weights).score(Xtrain, ytrain))
print("Accuracy of Tests without weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain).score(Xtest, ytest))
print("Accuracy of Tests with    weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain, sample_weight=weights).score(Xtest, ytest))

Accuracy of Train without weights 1.0
Accuracy of Train with    weights 1.0
Accuracy of Tests without weights 0.9554317548746518
Accuracy of Tests with    weights 0.9777158774373259


In [4]:
t = time.time()
for C in (.001, .01, 1, 5, 17):
    clf = Stree(C=C, random_state=random_state)
    clf.fit(Xtrain, ytrain)
    print(f"************** C={C} ****************************")
    print(f"Classifier's accuracy (train): {clf.score(Xtrain, ytrain):.4f}")
    print(f"Classifier's accuracy (test) : {clf.score(Xtest, ytest):.4f}")
    print(clf)
    print(f"**************************************************")
print(f"{time.time() - t:.4f} secs")

************** C=0.001 ****************************
Classifier's accuracy (train): 0.9749
Classifier's accuracy (test) : 0.9749
root
root - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([117]))
root - Up, <cgaf> - Leaf class=0 belief= 0.970833 counts=(array([0, 1]), array([699,  21]))

**************************************************
************** C=0.01 ****************************
Classifier's accuracy (train): 0.9797
Classifier's accuracy (test) : 0.9777
root
root - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([121]))
root - Up, <cgaf> - Leaf class=0 belief= 0.976257 counts=(array([0, 1]), array([699,  17]))

**************************************************
************** C=1 ****************************
Classifier's accuracy (train): 0.9869
Classifier's accuracy (test) : 0.9805
root
root - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([127]))
root - Up, <cgaf> - Leaf class=0 belief= 0.984507 counts=(ar

In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
scaler = StandardScaler()
cclf = CalibratedClassifierCV(base_estimator=LinearSVC(), cv=5)
cclf.fit(Xtrain, ytrain)
res = cclf.predict_proba(Xtest)
print(res[:4, :])

[[0.95399748 0.04600252]
 [0.92625258 0.07374742]
 [0.97804877 0.02195123]
 [0.94803313 0.05196687]]


In [6]:
#check iterator
for i in list(clf):
    print(i)

root
root - Down
root - Down - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([120]))
root - Down - Up, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([57]))
root - Up
root - Up - Down
root - Up - Down - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([14]))
root - Up - Down - Up, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([12]))
root - Up - Up, <cgaf> - Leaf class=0 belief= 0.993691 counts=(array([0, 1]), array([630,   4]))


In [7]:
#check iterator again
for i in clf:
    print(i)

root
root - Down
root - Down - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([120]))
root - Down - Up, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([57]))
root - Up
root - Up - Down
root - Up - Down - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([14]))
root - Up - Down - Up, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([12]))
root - Up - Up, <cgaf> - Leaf class=0 belief= 0.993691 counts=(array([0, 1]), array([630,   4]))


In [8]:
# Check if the classifier is a sklearn estimator
from sklearn.utils.estimator_checks import check_estimator
check_estimator(Stree())

In [9]:
# Make checks one by one
c = 0
checks = check_estimator(Stree(), generate_only=True)
for check in checks:
    c += 1
    print(c, check[1])
    check[1](check[0])

1 functools.partial(<function check_no_attributes_set_in_init at 0x128922b90>, 'Stree')
2 functools.partial(<function check_estimators_dtypes at 0x128918cb0>, 'Stree')
3 functools.partial(<function check_fit_score_takes_y at 0x128918b90>, 'Stree')
4 functools.partial(<function check_sample_weights_pandas_series at 0x1289144d0>, 'Stree')
5 functools.partial(<function check_sample_weights_not_an_array at 0x1289145f0>, 'Stree')
6 functools.partial(<function check_sample_weights_list at 0x128914710>, 'Stree')
7 functools.partial(<function check_sample_weights_shape at 0x128914830>, 'Stree')
8 functools.partial(<function check_sample_weights_invariance at 0x128914950>, 'Stree')
9 functools.partial(<function check_estimators_fit_returns_self at 0x12891ecb0>, 'Stree')
10 functools.partial(<function check_estimators_fit_returns_self at 0x12891ecb0>, 'Stree', readonly_memmap=True)
11 functools.partial(<function check_complex_data at 0x128914b00>, 'Stree')
12 functools.partial(<function check_dt